# Zero-Shot Prompting

In [2]:
import requests
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [4]:
from openai import OpenAI

In [6]:
requests.get("http://localhost:11434").content

b'Ollama is running'

In [8]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

In [10]:
model_name="llama3.2"

In [20]:
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
#from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_classic.chains import ConversationChain
from langchain_classic.memory import ConversationBufferMemory

In [26]:
# Initialize the Ollama model
llm = ChatOllama(
    model=model_name,   # change to mistral / qwen2.5 if you want
    temperature=0
)

def create_chain(prompt_template: str):
    """
    Create a LangChain runnable chain with the given prompt template.

    Args:
        prompt_template (str): The prompt template string.

    Returns:
        Runnable: A LangChain runnable sequence.
    """
    prompt = PromptTemplate.from_template(prompt_template)
    return prompt | llm

## Direct Task Specification

In [28]:
direct_task_prompt = """Classify the sentiment of the following text as positive, negative, or neutral.
Do not explain your reasoning, just provide the classification.

Text: {text}

Sentiment:"""

direct_task_chain = create_chain(direct_task_prompt)

# Test the direct task specification
texts = [
    "I absolutely loved the movie! The acting was superb.",
    "The weather today is quite typical for this time of year.",
    "I'm disappointed with the service I received at the restaurant."
]

for text in texts:
    result = direct_task_chain.invoke({"text": text}).content
    print(f"Text: {text}")
    print(f"Sentiment: {result}")

Text: I absolutely loved the movie! The acting was superb.
Sentiment: Positive
Text: The weather today is quite typical for this time of year.
Sentiment: Neutral.
Text: I'm disappointed with the service I received at the restaurant.
Sentiment: Negative


## Format Specification

In [31]:
format_spec_prompt = """Generate a short news article about {topic}. 
Structure your response in the following format:

Headline: [A catchy headline for the article]

Lead: [A brief introductory paragraph summarizing the key points]

Body: [2-3 short paragraphs providing more details]

Conclusion: [A concluding sentence or call to action]"""

format_spec_chain = create_chain(format_spec_prompt)

# Test the format specification prompting
topic = "The discovery of a new earth-like exoplanet"
result = format_spec_chain.invoke({"topic": topic}).content
print(result)

**Headline:** "Earth's Cousin Discovered: Scientists Stunned by New Exoplanet Find"

**Lead:** In a groundbreaking discovery that has sent shockwaves through the scientific community, astronomers have identified a new exoplanet that bears striking similarities to Earth. The planet, dubbed "Nyx-12," is located approximately 100 light-years from our solar system and boasts a surface temperature that could potentially support liquid water, a crucial ingredient for life.

**Body:** According to researchers at the European Southern Observatory (ESO), Nyx-12 orbits a stable G-type star, similar to the Sun, which suggests that it may have a habitable zone where conditions are suitable for life. The exoplanet's size and mass are also comparable to those of Earth, further fueling speculation about its potential for supporting life. Initial observations indicate that Nyx-12 has a thick atmosphere, with hints of oxygen and methane, which could be indicative of biological activity.

**Conclusion:*

## Multi-Step Reasoning

In [35]:
multi_step_prompt = """Analyze the following text for its main argument, supporting evidence, and potential counterarguments. 
Provide your analysis in the following steps:

1. Main Argument: Identify and state the primary claim or thesis.
2. Supporting Evidence: List the key points or evidence used to support the main argument.
3. Potential Counterarguments: Suggest possible objections or alternative viewpoints to the main argument.

Text: {text}

Analysis:"""

multi_step_chain = create_chain(multi_step_prompt)

# Test the multi-step reasoning approach
text = """While electric vehicles are often touted as a solution to climate change, their environmental impact is not as straightforward as it seems. 
The production of batteries for electric cars requires significant mining operations, which can lead to habitat destruction and water pollution. 
Moreover, if the electricity used to charge these vehicles comes from fossil fuel sources, the overall carbon footprint may not be significantly reduced. 
However, as renewable energy sources become more prevalent and battery technology improves, electric vehicles could indeed play a crucial role in combating climate change."""

result = multi_step_chain.invoke({"text": text}).content
print(result)

Here's the analysis of the text:

**1. Main Argument:**
The primary claim or thesis of the text is that electric vehicles (EVs) are not as environmentally friendly as they seem, and their impact on climate change is more complex than often touted.

**2. Supporting Evidence:**

* The production of batteries for EVs requires significant mining operations, leading to habitat destruction and water pollution.
* If the electricity used to charge EVs comes from fossil fuel sources, the overall carbon footprint may not be significantly reduced.
* However, as renewable energy sources become more prevalent and battery technology improves, EVs could play a crucial role in combating climate change.

**3. Potential Counterarguments:**

* One possible counterargument is that while mining operations for battery production can have negative environmental impacts, these can be mitigated through responsible sourcing practices, recycling, and closed-loop production methods.
* Another potential counterarg

## Comparative Analysis

In [38]:
def compare_prompts(task, prompt_templates):
    """
    Compare different prompt templates for the same task.
    
    Args:
        task (str): The task description or input.
        prompt_templates (dict): A dictionary of prompt templates with their names as keys.
    """
    print(f"Task: {task}\n")
    for name, template in prompt_templates.items():
        chain = create_chain(template)
        result = chain.invoke({"task": task}).content
        print(f"{name} Prompt Result:")
        print(result)
        print("\n" + "-"*50 + "\n")

task = "Explain concisely the concept of blockchain technology"

prompt_templates = {
    "Basic": "Explain {task}.",
    "Structured": """Explain {task} by addressing the following points:
1. Definition
2. Key features
3. Real-world applications
4. Potential impact on industries"""
}

compare_prompts(task, prompt_templates)

Task: Explain concisely the concept of blockchain technology

Basic Prompt Result:
Blockchain technology is a decentralized, digital ledger that records transactions across a network of computers in a secure and transparent manner. It uses cryptography to ensure the integrity and immutability of data, allowing for:

1. Secure storage: Data is stored on multiple nodes, making it difficult to manipulate or alter.
2. Transparency: All transactions are recorded publicly, providing a clear history of activity.
3. Consensus: Nodes agree on the state of the blockchain through complex algorithms.

This creates a tamper-proof and decentralized system for storing and sharing data, enabling applications such as cryptocurrency, supply chain management, and identity verification.

--------------------------------------------------

Structured Prompt Result:
Here's a concise explanation of blockchain technology:

**Definition:** Blockchain is a decentralized, digital ledger that records transactions

# Few-Shot Learning

In [41]:
def few_shot_sentiment_classification(input_text):
    few_shot_prompt = PromptTemplate(
        input_variables=["input_text"],
        template="""
        Classify the sentiment as Positive, Negative, or Neutral.
        
        Examples:
        Text: I love this product! It's amazing.
        Sentiment: Positive
        
        Text: This movie was terrible. I hated it.
        Sentiment: Negative
        
        Text: The weather today is okay.
        Sentiment: Neutral
        
        Now, classify the following:
        Text: {input_text}
        Sentiment:
        """
    )
    
    chain = few_shot_prompt | llm
    result = chain.invoke(input_text).content

    # Clean up the result
    result = result.strip()
    # Extract only the sentiment label
    if ':' in result:
        result = result.split(':')[1].strip()
    
    return result  # This will now return just "Positive", "Negative", or "Neutral"

test_text = "I can't believe how great this new restaurant is!"
result = few_shot_sentiment_classification(test_text)
print(f"Input: {test_text}")
print(f"Predicted Sentiment: {result}")

Input: I can't believe how great this new restaurant is!
Predicted Sentiment: Positive


In [44]:
def multi_task_few_shot(input_text, task):
    few_shot_prompt = PromptTemplate(
        input_variables=["input_text", "task"],
        template="""
        Perform the specified task on the given text.
        
        Examples:
        Text: I love this product! It's amazing.
        Task: sentiment
        Result: Positive
        
        Text: Bonjour, comment allez-vous?
        Task: language
        Result: French
        
        Now, perform the following task:
        Text: {input_text}
        Task: {task}
        Result:
        """
    )
    
    chain = few_shot_prompt | llm
    return chain.invoke({"input_text": input_text, "task": task}).content

print(multi_task_few_shot("I can't believe how great this is!", "sentiment"))
print(multi_task_few_shot("Guten Tag, wie geht es Ihnen?", "language"))

The sentiment of the text "I can't believe how great this is!" is Positive.
The text "Guten Tag, wie geht es Ihnen?" is in German.


## In-Context Learning

In [47]:
def in_context_learning(task_description, examples, input_text):
    example_text = "".join([f"Input: {e['input']}\nOutput: {e['output']}\n\n" for e in examples])
    
    in_context_prompt = PromptTemplate(
        input_variables=["task_description", "examples", "input_text"],
        template="""
        Task: {task_description}
        
        Examples:
        {examples}
        
        Now, perform the task on the following input:
        Input: {input_text}
        Output:
        """
    )
    
    chain = in_context_prompt | llm
    return chain.invoke({"task_description": task_description, "examples": example_text, "input_text": input_text}).content

task_desc = "Convert the given text to pig latin."
examples = [
    {"input": "hello", "output": "ellohay"},
    {"input": "apple", "output": "appleay"}
]
test_input = "python"

result = in_context_learning(task_desc, examples, test_input)
print(f"Input: {test_input}")
print(f"Output: {result}")

Input: python
Output: def convert_to_pig_latin(word):
    if word[0] in 'aeiou':
        return word + 'y'
    else:
        return word[1:] + word[0] + 'ay'

print(convert_to_pig_latin('python'))  # Output: ythonpay


## Evaluation

In [50]:
def evaluate_model(model_func, test_cases):
    '''
    Evaluate the model on a set of test cases.

    Args:
    model_func: The function that makes predictions.
    test_cases: A list of dictionaries, where each dictionary contains an "input" text and a "label" for the input.

    Returns:
    The accuracy of the model on the test cases. 
    '''
    correct = 0
    total = len(test_cases)
    
    for case in test_cases:
        input_text = case['input']
        true_label = case['label']
        prediction = model_func(input_text).strip()
        
        is_correct = prediction.lower() == true_label.lower()
        correct += int(is_correct)
        
        print(f"Input: {input_text}")
        print(f"Predicted: {prediction}")
        print(f"Actual: {true_label}")
        print(f"Correct: {is_correct}\n")
    
    accuracy = correct / total
    return accuracy

test_cases = [
    {"input": "This product exceeded my expectations!", "label": "Positive"},
    {"input": "I'm utterly disappointed with the service.", "label": "Negative"},
    {"input": "The temperature today is 72 degrees.", "label": "Neutral"}
]

accuracy = evaluate_model(few_shot_sentiment_classification, test_cases)
print(f"Model Accuracy: {accuracy:.2f}")

Input: This product exceeded my expectations!
Predicted: Positive
Actual: Positive
Correct: True

Input: I'm utterly disappointed with the service.
Predicted: Negative
Actual: Negative
Correct: True

Input: The temperature today is 72 degrees.
Predicted: Sentiment
Actual: Neutral
Correct: False

Model Accuracy: 0.67


# Basic Chain-of_Thoughts Prompting

In [53]:
# Standard prompt
standard_prompt = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question concisely: {question}."
)

# Chain of Thought prompt
cot_prompt = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question step by step concisely: {question}"
)

# Create chains
standard_chain = standard_prompt | llm
cot_chain = cot_prompt | llm

# Example question
question = "If a train travels 120 km in 2 hours, what is its average speed in km/h?"

# Get responses
standard_response = standard_chain.invoke(question).content
cot_response = cot_chain.invoke(question).content

print("Standard Response:")
print(standard_response)
print("\nChain of Thought Response:")
print(cot_response)

Standard Response:
To find the average speed, divide the distance traveled (120 km) by the time taken (2 hours).

Average speed = Distance / Time
= 120 km / 2 hours
= 60 km/h.

Chain of Thought Response:
Here's the solution:

1. Distance traveled = 120 km
2. Time taken = 2 hours
3. Average speed = Total distance / Total time
4. Average speed = 120 km / 2 hours
5. Average speed = 60 km/h


In [55]:
advanced_cot_prompt = PromptTemplate(
    input_variables=["question"],
    template="""Solve the following problem step by step. For each step:
1. State what you're going to calculate
2. Write the formula you'll use (if applicable)
3. Perform the calculation
4. Explain the result

Question: {question}

Solution:"""
)

advanced_cot_chain = advanced_cot_prompt | llm

complex_question = "A car travels 150 km at 60 km/h, then another 100 km at 50 km/h. What is the average speed for the entire journey?"

advanced_cot_response = advanced_cot_chain.invoke(complex_question).content
print(advanced_cot_response)

Here's the step-by-step solution to the problem:

**Step 1: State what we're going to calculate**
We want to find the average speed for the entire journey.

**Step 2: Write the formula we'll use (if applicable)**
The formula for average speed is:

Average Speed = Total Distance / Total Time

Since we don't know the total time, we can also use the formula:

Total Time = (Distance1 / Speed1) + (Distance2 / Speed2)

**Step 3: Perform the calculation**

First, let's calculate the time taken for each part of the journey:

Time taken to travel 150 km at 60 km/h:
Time = Distance / Speed
= 150 km / 60 km/h
= 2.5 hours

Time taken to travel 100 km at 50 km/h:
Time = Distance / Speed
= 100 km / 50 km/h
= 2 hours

Now, let's calculate the total time:
Total Time = (Distance1 / Speed1) + (Distance2 / Speed2)
= (150 km / 60 km/h) + (100 km / 50 km/h)
= 2.5 hours + 2 hours
= 4.5 hours

**Step 4: Explain the result**
The total time taken for the entire journey is 4.5 hours, and the total distance trav

## Comparative Analysis

In [58]:
challenging_question = """
A cylindrical water tank with a radius of 1.5 meters and a height of 4 meters is 2/3 full. 
If water is being added at a rate of 10 liters per minute, how long will it take for the tank to overflow? 
Give your answer in hours and minutes, rounded to the nearest minute. 
(Use 3.14159 for π and 1000 liters = 1 cubic meter)"""

standard_response = standard_chain.invoke(challenging_question).content
cot_response = advanced_cot_chain.invoke(challenging_question).content

print("Standard Response:")
print(standard_response)
print("\nChain of Thought Response:")
print(cot_response)

Standard Response:
To find the time it takes for the tank to overflow, we need to calculate the volume of water in the tank and then divide that by the rate at which water is being added.

The volume of a cylinder is given by V = πr^2h, where r is the radius and h is the height. 

First, let's find the total capacity of the tank:

V = π(1.5)^2(4) 
= 3.14159 * 2.25 * 4
≈ 28.27 cubic meters

Since the tank is 2/3 full, we need to calculate the volume of water in the tank:

Volume of water = (2/3) * 28.27 ≈ 18.755 cubic meters

Now, let's convert this volume from cubic meters to liters:

1 cubic meter = 1000 liters
So, 18.755 cubic meters = 18,755 liters

The rate at which water is being added is 10 liters per minute. To find the time it takes for the tank to overflow, we can divide the total volume of water by this rate:

Time = Volume / Rate 
= 18,755 / 10
≈ 1875 minutes 

Now, let's convert this time from minutes to hours and minutes:

1875 minutes ≈ 31.25 hours

So, it will take appro

## Problem-Solving Applications

In [61]:
logical_reasoning_prompt = PromptTemplate(
    input_variables=["scenario"],
    template="""Analyze the following logical puzzle thoroughly. Follow these steps in your analysis:

List the Facts:

Summarize all the given information and statements clearly.
Identify all the characters or elements involved.
Identify Possible Roles or Conditions:

Determine all possible roles, behaviors, or states applicable to the characters or elements (e.g., truth-teller, liar, alternator).
Note the Constraints:

Outline any rules, constraints, or relationships specified in the puzzle.
Generate Possible Scenarios:

Systematically consider all possible combinations of roles or conditions for the characters or elements.
Ensure that all permutations are accounted for.
Test Each Scenario:

For each possible scenario:
Assume the roles or conditions you've assigned.
Analyze each statement based on these assumptions.
Check for consistency or contradictions within the scenario.
Eliminate Inconsistent Scenarios:

Discard any scenarios that lead to contradictions or violate the constraints.
Keep track of the reasoning for eliminating each scenario.
Conclude the Solution:

Identify the scenario(s) that remain consistent after testing.
Summarize the findings.
Provide a Clear Answer:

State definitively the role or condition of each character or element.
Explain why this is the only possible solution based on your analysis.
Scenario:

{scenario}

Analysis:""")

logical_reasoning_chain = logical_reasoning_prompt | llm

logical_puzzle = """In a room, there are three people: Amy, Bob, and Charlie. 
One of them always tells the truth, one always lies, and one alternates between truth and lies. 
Amy says, 'Bob is a liar.' 
Bob says, 'Charlie alternates between truth and lies.' 
Charlie says, 'Amy and I are both liars.' 
Determine the nature (truth-teller, liar, or alternator) of each person."""

logical_reasoning_response = logical_reasoning_chain.invoke(logical_puzzle).content
print(logical_reasoning_response)

**List the Facts:**

1. There are three people in a room: Amy, Bob, and Charlie.
2. One person always tells the truth, one always lies, and one alternates between truth and lies.
3. Amy says, 'Bob is a liar.'
4. Bob says, 'Charlie alternates between truth and lies.'
5. Charlie says, 'Amy and I are both liars.'

**Identify all the characters or elements involved:**

1. Amy
2. Bob
3. Charlie

**Possible Roles or Conditions:**

1. Truth-teller (always tells the truth)
2. Liar (always lies)
3. Alternator (alternates between truth and lies)

**Note the Constraints:**

1. The statements made by each person must be consistent with their role.
2. If Amy is a truth-teller, Bob cannot be a liar, as Amy's statement would be false.
3. If Charlie is an alternator, his statement about Amy and himself being liars could be either true or false.

**Generate Possible Scenarios:**

1. Amy is the Truth-Teller:
	* Bob is the Liar
	* Charlie is the Alternator
2. Amy is the Liar:
	* Bob is the Truth-Teller
	